In [0]:

%run ../00_Config

In [0]:
# Load the Brightspace sample data
file_path = "/Workspace/Users/pradeep.ponati@gmail.com/samplefiles/brightspace_sample_data.csv"





In [0]:
# Read CSV into DataFrame
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

In [0]:
# Show basic info like count of records, also some sample data 
print(f"✓ Loaded {df.count()} records")
print("\nSchema:")
df.printSchema()

print("\nSample data:")
df.show(10, truncate=False)

In [0]:
display(df)

In [0]:
# Use your ADLS Gen2 storage account
storage_account = "streamingstorageb"
container = "bronze"  # We'll use a container called 'bronze'

# Define Bronze table path in ADLS Gen2
bronze_path = BRONZE_BRIGHTSPACE

print(f"Target path: {bronze_path}")


In [0]:

# Add metadata columns
from pyspark.sql.functions import current_timestamp, current_date, lit, md5, concat_ws

df_bronze = df \
    .withColumn("_ingestion_timestamp", current_timestamp()) \
    .withColumn("_ingestion_date", current_date()) \
    .withColumn("_source_system", lit("brightspace_lms")) \
    .withColumn("_record_hash", md5(concat_ws("|", *df.columns)))

print(f"\n📝 Writing to ADLS Gen2...")
print(f"   Storage Account: {storage_account}")
print(f"   Container: {container}")
print(f"   Records: {df_bronze.count()}")

try:
    # Write to Delta Lake on ADLS Gen2
    df_bronze.write \
        .format("delta") \
        .mode("overwrite") \
        .save(bronze_path)
    
    print(f"\n✓ Bronze table created successfully!")
    print(f"✓ Location: {bronze_path}")
    
    # Verify
    bronze_df = spark.read.format("delta").load(bronze_path)
    print(f"✓ Verification: Read back {bronze_df.count()} records")
    
except Exception as e:
    print(f"\n⚠️ Error: {str(e)}")
    print("\nThis might mean:")
    print("1. Container 'bronze' doesn't exist in your storage account")
    print("2. Authentication not configured")
    print("\nLet me know what error you see and we'll fix it!")